# Introduction to programming 4

William Vigor, Clyde Fare and João Pedro Malhado, Imperial College London (contact: [chemistry-git@imperial.ac.uk](mailto:chemistry-git@imperial.ac.uk))

Notebook is licensed under a [Creative Commons Attribution 4.0 (CC-by) license](http://creativecommons.org/licenses/by/4.0/).

## Overview

Up to now all our programs have run in their little bubble: they define the variable values they use and after they finish they leave little trace that they were ever run. Things changed a little in the last workshop where we introduced the possibility of the user passing on some information for the program to process. We will go a step further in seeing how we can interact with files.

Dealing and interacting with files is a very common way of processing information stored in a computer, and is something that your programs might often do. When we deal with files and our programs go beyond their "logical bubble", is when we can start doing some damage: delete information on files, delete files all together, fill up the hard disk, damage your installation (the operating system should actually have safeguards for this). This should not intimidate you, it is just that with power it comes responsibility.

In this workshop we will first go over the basic operations on files. Then we will look at a case study of a common scenario where we want to process a file to extract information in a useful format.

## Workshop content

#### Part 1
* [Dealing with files](#files)
* [Parsing a simple XYZ file - exercise](#xyz)

#### Part 2
* [Parsing a complex file - guided exercise](#dimers)

## Dealing with files <a id="files"></a>

In any computer hard-disk there are typically two types of files: text files and binary files. Text files are those you can read with a text editor program (think of Notepad, not Microsoft Word), and do not necessarily contain prose. (If you open this notebook, which has an extension \*.ipynb, on a text editor you will see that it is text in some form. The Jupyter notebook is a text file.) Binary files on the other end cannot be opened on a text editor, or when they do open they show up as ununderstandable characters. (Picture files like PNG or JPEG, music files like MP3, video files, Word DOC files, are all binary files.)

We will only be looking at manipulating text files. Although Python can also manipulate binary files, the caveat of such files is that the programmer needs to know *a priori* how the content of the file is structured, and since one cannot just look at the file on a text editor, this task is harder. Binary files further require knowledge of some low level computer architecture which is beyond the scope of this course. Manipulating text files will however illustrate the process, and most instruments and computer programs are able to write data in some form of text file.

We will start by loading the content of the file <a href="test.out">test.out</a> (you can open the file to look at its content). First we need to open a stream to the file content and set it to a variable. This is done with function *open()*

    stream1=open('test.out','r')

This takes a string containing the name of our file (including the full path if the file is not in the same directory as the notebook/script) and as a second argument a flag telling Python whether we are going to read or write to the file. ('r' for reading and 'w' for writing).

Via the variable *stream1* we now have direct access to the content of the file. If we loop over the file stream we can read the lines of the file one by one as strings

    for line in stream1:
        print(line)

Once we are done with the file we should tidy up, close the stream and leave the file in peace

    stream1.close()

If we are interested in a list with every line in the file we could build a list with a loop, or simply operate with the function *list()* on the stream

    stream2=open('test.out','r')
    list_content=list(stream2)
    stream2.close()
    list_content

Note at the end of each line/list element the linebreak represented by '\n' which counts as a single character.

We should be slightly careful when performing this operations as it puts all the content of the file into a list, which will be unmanageable if the file is several gigabytes in size.

It is just as simple to read a specific number of lines, using a loop and the method *.readline()*

    stream3=open('test.out','r')
    for i in range(2):
        line=stream3.readline()
        print(line)
    stream3.close()

The house keeping step of closing the stream can be a bit tedious an easily forgotten. The following construct will close the stream for us when we are done

    with open('test.out','r') as stream:
        list_content_again=list(stream)
    list_content_again

If we are just interested in some specific information in the file we can search for it as we read it.

The following code looks for the line with the substring 'leaves' and extracts the colour of the leaves in the text.

In [ ]:
with open('test.out','r') as stream:
    for line in stream:
        if 'leaves' in line:
            leaf_colour = line.split()[4]
        
leaf_colour[:-1]

Note that the variable *line* is a string with each line in the file. If we are interested in individual words, we can form a list from a string with the **.split()** method.

    "Read my words. One by one.".split()

By default *.split()* separates the string on the blank spaces (space or tab characters), but we can choose any other character

    "Read my words. One by one.".split(".")

For completeness, the opposite operation to *.split()* is performed by *.join()*

    "--".join(['three','two','one','go'])

Is it clear how we are obtaining the colour of the leaves? Write some code below that extracts the colour of the sky instead.

The authors of those words were probably not in their best mood. So we are going to change the text to make it more cheerful (even if slightly psychedelic). The goal is thus to construct a list of the verses in the lyrics, but with *blue* leaves, a *bay* sky and a *glorious* day. Let us call this list *cheerful*.

This task could be done in one go with a single *for* loop (if you are feeling comfortable you can try to implement such solution). We will however break the task into two. First, create the list *cheerful* where each element is a verse of the original text split into words.

Now change your list such that you replace the wanted words in the text, and put each verse together using the *.join()* method such that *cheerful* is a list of strings with the verses of the lyrics.

If we want to write our cheerful version to a file, we just do

    with open('cheerful.out','w') as stream:
        stream.writelines(cheerful)

Note that if you open a file for writing you **will overwrite whatever was initially in the file**.

The [os module](https://docs.python.org/3/library/os.html") provides many functions for interaction with the operating system, including the file system. We can get the list of the files in the current directory and see if our new created file is in place

    import os
    os.listdir()

### XYZ molecular structure files <a id="xyz"></a>

A very common type of text file used in chemistry is the <a href="http://openbabel.org/wiki/XYZ_(format)">XYZ file</a> to represent chemical structures. The <a href="H2S.xyz">H2S.xyz</a> file, present in the same directory as the notebook, is an example of such file (you can open it using a molecular viewer program such as Avogadro). If we open the H2S.xyz file on a text editor, we can clearly note the general structure of such files:

* The first line of the file is formed by a single integer number *n* indicating the number atoms present in the chemical structure in the file.
* The second line is a string containing a comment about the file. It is often left blank, but this line must be present.
* It follows *n* lines with the chemical symbol of each atom and its position in space in Cartesian coordinates in Ångström.

In this exercise, we will want to determine what is the bond angle of the H<sub>2</sub>S molecule represented in the H2S.xyz.

First, write some code to extract the position of each atom in the molecule, in the form of a list where each element is another list with the atom coordinates.

Note that the bond angle can be determined with relative ease, by thinking of the vectors that connect the atom positions:

<img src="molecule_angle.svg" style="width:30%" />

Using arrays from the Numpy module, obtain the two relevant vectors from the atoms' coordinates previously taken from the file.

Using the <a href="http://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html">dot()</a> and <a href="http://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.norm.html">norm()</a> functions from the Numpy module, determine the bond angle on the H<sub>2</sub>S molecule.

# Part 2

## Ne<sub>2</sub> dissociation: extracting information from files <a id="dimers"></a>

In this mini-project we will be looking at the dissociation curve of two neon atoms, i.e. how their electronic energy varies with the distance between the two atoms. By analysing this curve, beyond other quantities, it is possible to determine the equilibrium distance between atoms at low temperature.

This task will involve extracting the relevant information from the output file of a quantum chemistry calculation. The file <a href="Ne.out">Ne.out</a> is a fairly typical log file of an electronic structure calculation program, in this case from the <a href="http://www.psicode.org">Psi4</a> program, and corresponds to a calculation of the electronic energy of a single neon atom. By looking at the file one notices that a lot of detail is given about the electronic structure of neon and about the calculation itself. We want to write a program that extracts from the file the relevant information to accomplish the task in hand.

The situation where we have a text file generated by a program or an instrument, and wish to extract some relevant information (for example to make a plot), is a fairly common one both when working in an experimental or computational setting. We will look at how to address this problem.

We will first focus our attention on the energy of an isolated neon atom. We can find this energy expressed in <a href="http://physics.nist.gov/cgi-bin/cuu/Value?hrj">Hartrees</a> towards the end of the Ne.out file on the line

      Total Energy              =    -128.6763534151176600 [H]

Note that the substring "Total Energies" shows up several times in the file, so it is important to search for a substring with the correct number of spaces.
In the cell below remove 'pass' and in its place write some code that will read the log file and set a variable named 'energy_str' to the string with the energy of the neon atom

In [ ]:
with open('Ne.out', 'r') as f:
    for line in f:
        pass

Now we have extracted the energy as a string, modify the code to generate a new variable named 'ne_energy' which contains the energy as a floating point number.

In [ ]:
with open('Ne.out', 'r') as f:
    for line in f:
        pass

Based on the code above, define a function *get_atom_energy()* which receives a string with the name of a file of an atom electronic structure calculation and returns the MP2 total energy from that file.

We now have a way to extract the energy of a single atom, but we are interested in the energy of 2 atoms as a function of the energy. The file <a href="Ne2.out">Ne2.out</a> contains energies for a neon dimer for different positions of the two atoms.

You can open the file and note that is has the following format:

This format is then repeated giving data for several different coordinates of neon atom pairs. Our task is to extract both the information about atom position (from which we will obtain the distance) and the energy. We will tackle these two problems separately at first.

The coordinates of the two Ne atoms are specified in Å:

       Center              X                  Y                   Z
    ------------   -----------------  -----------------  -----------------
          NE          0.000000000000     0.000000000000    -0.250000000000
          NE          0.000000000000     0.000000000000     0.250000000000

Unfortunately the two lines have the same format so it is not quite as easy as before to distinguish between lines and extract the data. But we do know that whenever we find a 'Ne' symbol on the line we want to extract the coordinates, but we need a mechanism to distinguish between atom 1 and atom 2.

Study the following cell. In it the code goes through the file line by line, and prints out the neon coordinates specifying which neon atom the coordinates refer to:

In [ ]:
#we use the variable first_Ne as a flag to indicate whether the next line with an 'Ne' in it
#will be the first or the second Neon atom. It is initiated to True
first_Ne = True

with open('Ne2.out', 'r') as f:
    for line in f:
        #if we are on the line with the first Ne
        if 'NE' in line.split() and first_Ne:
            print('1: ' + line)
        
            #set the flag to false because the next line with Ne on will be the second Ne
            first_Ne=False
        
        #we are on the line with second Ne
        elif 'NE' in line.split() and not first_Ne:
            print('2: ' + line)

            #set the flag to true because the next line will Ne on with be the first Ne
            first_Ne=True

Modify the cell below so that instead of printing the whole line out every time we find Neon coordinates, instead we just print out a list with the x,y and z coordinates.

In [ ]:
first_Ne= True

with open('Ne2.out','r') as f:
    for line in f:
        #we are on the line with the first Ne
        if 'NE' in line.split() and first_Ne:

            #set the flag to false because the next line with Ne on will be the second Ne
            first_Ne=False
            
        #we are on the line with second Ne
        elif 'NE' in line.split() and not first_Ne:

            #set the flag to true because the next line with Ne on will be the first Ne
            first_Ne=True

OK, the task now is to generate a list of distances between the Neon atoms in the dimers. Above you have found a method for distinguishing between the first and second atoms in each dimer and have used it to print out the coordinates of each atom. Below, the function *vect_dist()* calculates the distance between two atoms given the coordinates of each in Cartesian coordinates. (This function could have been defined using arrays, but here we take a more explicit approach)

In [ ]:
def vect_dist(x1, y1, z1, x2, y2, z2):
    distance = ( (x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2 )**0.5
    return(distance)

Adapt the code we wrote before such that we call the *vect_dist()* every time we find the second neon atom, and in this way build a list *dist* with all distances between neon atoms.

In [ ]:
dist=[]


Here is a good point to check your answer. The function *check_float()* checks if two numbers are equal with a tolerance of 1&times;10<sup>-8</sup>.

In [ ]:
def check_float(float_number, exact_answer):
    tol = 1e-8
    return abs(float_number - exact_answer) < tol

Run the cell bellow and check the output is the same:

In [ ]:
correct_answer = [2.3, 2.35, 2.40, 2.45, 2.5, 2.55, 2.60, 2.65, 2.70, 2.75, 2.80, 2.95, 3.00, 3.05 , 3.10, 3.15, 3.20, 3.25, 3.3, 3.35]

for i,d in enumerate(dist):
    if check_float(d, correct_answer[i]):
        print('Success')
    else:
        print("The lists don't match, check your above solutions.")

We have now successfully extracted the distances between neon atoms from the log file. The next step is to also extract the energies (we already addressed this problem when extracting the energy of a single neon).

Adapt the code you wrote before such that you build both the *dist* and *energies* lists with a single loop through the file.

In [ ]:
dist=[]
energies=[]



Again here is a good point to check you answer. Run the cell below and check the output is the same:

In [ ]:
correct_answer = [-257.3508759806243802,-257.3515014203481996,-257.3519632198754152,-257.3522960563400943,-257.3525285912639902,-257.3526844311371633,-257.3527828598416818,-257.3528394280590987,-257.3528664678370887,-257.3528735535761029,-257.3528679338846814,-257.3528206560993112,-257.3528035490451771,-257.3527879372218194,-257.3527742712665258,-257.3527626771479504,-257.3527530774693446,-257.3527452802550215,-257.3527390409376494,-257.3527341032995537]

for i,d in enumerate(energies):
    if check_float(d, correct_answer[i]):
        print('Success')
    else:
        print("The lists don't match, check your above solutions.")

        
correct_answer = [2.3, 2.35, 2.40, 2.45, 2.5, 2.55, 2.60, 2.65, 2.70, 2.75, 2.80, 2.95, 3.00, 3.05 , 3.10, 3.15, 3.20, 3.25, 3.3, 3.35]

for i,d in enumerate(dist):
    if check_float(d, correct_answer[i]):
        print('Success')
    else:
        print("The lists don't match, check your above solutions.")

Let us now make our solution reusable by defining a function *get_dimer_distance_energy()*, that should receive two strings as arguments: a file name, chemical symbol search pattern. The function should return a list where the first element is a list of distances and the second element a list of energies.

## Ne<sub>2</sub> dissociation: analysing the result

Now that we created a mechanism for extracting the data from the text file, let us use it to do some interesting things. First we will plot the data, using the standard [matplotlib](https://matplotlib.org/users/pyplot_tutorial.html) python plotting module, to find the characteristic diatomic energy pattern.

In [ ]:
from matplotlib.pyplot import *

ne2_data=get_dimer_distance_energy('Ne2.out','NE')
dist=ne2_data[0]
energy=ne2_data[1]

plot(dist, energy, marker='+')
xlabel('Ne - Ne Bond Distance ($\AA$)')
ylabel('Energy (Hartree)')
show()

In chemistry we are more interested in the interaction energy of the two atoms, i.e. the relative energy with respect to 2 separate atoms in space. Subtract twice the energy of a free Ne atom (which we can obtain via the get_atom_energy() function) and plot how interaction energy varies with distance. (It is convenient here to work with arrays instead of lists).

This shape of dissociation curve can be modeled mathematically by the Morse potential:

$$V(r) = d ((1 - e^{-a(r - r_e)})^2 - 1),$$

where *r<sub>e</sub>* is the equilibrium distance, *d* is the dissociation energy, and the *a* is related to the frequency of the potential well.

Write a function to compute the Morse potential with *r*, *r<sub>e</sub>*, *d* and *a* as arguments to the function, making sure the arguments are defined in this order.

Run the cell below to check if your function is defined correctly.

In [ ]:
if check_float(morse(2.5, 1.0, 3.2, 5.2) , -0.002621766640661 ):
    print('Well Done your Morse function works.')
else:
    print('Your Morse function is wrong!')

We are interested on the equilibrium distance and dissociation energy of the neon dimer. In order to obtain these we can use the function *curve_fit()* from the scipy.optimize module. (You may want to revise how to do this from you 1st year course notebooks: if you don't have them in an accessible place you can get them from <a href="https://github.com/imperialchem/python-data-viz-intro/tree/master/data_workshop4">here</a>).

In the process of fitting you will want to pay attention to the initial parameter guess. How can you extract good guesses for *r<sub>e</sub>* and *d* from the data?

In [ ]:
from scipy.optimize import curve_fit



To check that the fit makes sense. Plot the fitted curve along with original data (to get a smooth curve we should use the function linspace() to plot the fitted curve).

## Extra: Trends along the group

You will find on the same directory as the notebook, besides the log file for the Ne dimer, also log files for He, Ar and Kr. It is interesting to observe the trend of equilibrium distance and dissociation energy along the group. In order to do this it is useful that we automatise the analysis done above.

We shall define two functions, *eqdist_dissenergy()* and *disscurve()* both receiving three string arguments: the file name with the energy of a single atom, the file name for a file containing data for the dimer at different distances, and a chemical element search string. The *eqdist_dissenergy()* should call *get_dimer_distance_energy()* and *get_atom_energy()*, fit the data to a Morse potential (it should provide adequate guesses for the fit), and return the values of the equilibrium distance and dissociation energy.

Because fitting blindly is a dangerous thing to do, we should define *disscurve()* which should do most of the same work as *eqdist_dissenergy()* but show us instead the plot of the fitted dissociation curve and original data.

Use the functions defined above to plot the variation of dimer equilibrium distance and dissociation energy along the noble gas group.

## Summary

Files are an important way of permanent storage of data. Handling files is important not only for processing data generated by instruments or other programs, but also to store results generated by your own programs.

In this workshop we have seen how to access data in files files. The function *open()* creates a stream to access the file content, which can be looped line by line. Each line is retrieved as a string, in which context the string method *.split()* becomes useful for further processing.

We used file processing techniques to extract the data of the dissociation curve of a dimer from a complex file. We further analysed this data to obtain the dissociation energy and equilibrium distance of the dimer.

# If you want to go further

In these four workshops we covered most of the basic concepts in computer programming which are common to many programming languages. You can go a long way with just what is covered here, but if you want to go further there is a lot to look towards to. The following is a brief list of online resources.

* [The Official Python Documentation](http://docs.python.org), including
    * [Python Tutorial](http://docs.python.org/3/tutorial),
    * [Python Language Reference](http://docs.python.org/3/reference) with documentation on all commands and features of the Python language.
* [The Code Academy Python course](http://www.codecademy.com/en/tracks/python)
* [Learn Python The Hard Way](http://learnpythonthehardway.org/book/)
* [Dive Into Python](http://www.diveintopython.net/)